In [1]:
import numpy as np
from asdf import AsdfFile
from astropy.io import fits
from astropy import wcs as astwcs
from astropy import table
from gwcs import wcs

from jwst import datamodels
from jwst.assign_wcs import nirspec
from jwst.transforms import models

In [2]:
# These are the CV3 files
# It uses the new ifupost and ote files
refs = {'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf',
        'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf',
        'disperser': '/grp/crds/cache/references/jwst/jwst_nirspec_disperser_0035.asdf',
        'distortion': 'N/A',
        'filteroffset': 'N/A',
        'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf',
        'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf',
        'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf',
        'ifupost': 'ifupost.asdf',
        'ifuslicer': '/grp/crds/cache/references/jwst/jwst_nirspec_ifuslicer_0003.asdf',
        'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf',
        'ote': 'ote.asdf',
        'regions': 'N/A',
        'specwcs': 'N/A',
        'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf'}


In [3]:
wcs_kw = {'wcsaxes': 2, 'ra_ref': 165, 'dec_ref': 54,
          'v2_ref': -8.3942412, 'v3_ref': -5.3123744, 'roll_ref': 37,
          'crpix1': 1024, 'crpix2': 1024,
          'cdelt1': .08, 'cdelt2': .08,
          'ctype1': 'RA---TAN', 'ctype2': 'DEC--TAN',
          'pc1_1': 1, 'pc1_2': 0, 'pc2_1': 0, 'pc2_2': 1
          }

slit_fields_num = ["shutter_id", "xcen", "ycen",
                   "ymin", "ymax", "quadrant", "source_id",
                   "stellarity", "source_xpos", "source_ypos"]


slit_fields_str = ["name", "shutter_state", "source_name", "source_alias"]


In [4]:
t = table.Table.read('/home/dencheva/data/astropy/test-nrs/nrs_testing/test_model/ifu_grating_functional_ESA_v1_20180619.txt', format='ascii')
#t.columns

In [5]:
def create_hdul(detector='NRS1'):
    """
    Create a fits HDUList instance.
    """
    hdul = fits.HDUList()
    phdu = fits.PrimaryHDU()
    phdu.header['instrume'] = 'NIRSPEC'
    phdu.header['detector'] = detector
    phdu.header['time-obs'] = '8:59:37'
    phdu.header['date-obs'] = '2016-09-05'

    scihdu = fits.ImageHDU()
    scihdu.header['EXTNAME'] = "SCI"
    for item in wcs_kw.items():
        scihdu.header[item[0]] = item[1]
    hdul.append(phdu)
    hdul.append(scihdu)
    return hdul

In [6]:
def create_nirspec_ifu_file(filter='F290LP', grating='G395H', lamp='N/A', detector='NRS1'):
    image = create_hdul(detector)
    image[0].header['exp_type'] = 'NRS_IFU'
    image[0].header['filter'] = filter
    image[0].header['grating'] = grating
    image[1].header['crval3'] = 0
    image[1].header['wcsaxes'] = 3
    image[1].header['ctype3'] = 'WAVE'
    image[0].header['lamp'] = lamp
    image[0].header['GWA_XTIL'] = 0.35986012
    image[0].header['GWA_YTIL'] = 0.13448857
    return image

In [7]:
!ulimit -n 1024
hdul = create_nirspec_ifu_file(grating='G395H', filter='F290LP',  detector='NRS1')
im = datamodels.ImageModel(hdul)

pipeline = nirspec.create_pipeline(im, refs)
w = wcs.WCS(pipeline)
im.meta.wcs = w

2018-06-20 10:59:19,794 - stpipe - INFO - gwa_ytilt is 0.13448857 deg
2018-06-20 10:59:19,795 - stpipe - INFO - gwa_xtilt is 0.35986012 deg
2018-06-20 10:59:19,796 - stpipe - INFO - theta_y correction: -0.041645342310905185 deg
2018-06-20 10:59:19,798 - stpipe - INFO - theta_x correction: 0.0 deg
2018-06-20 10:59:32,696 - stpipe - INFO - Created a NIRSPEC nrs_ifu pipeline with references {'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf', 'disperser': '/grp/crds/cache/references/jwst/jwst_nirspec_disperser_0035.asdf', 'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf', 'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf', 'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf', 'filteroffset': 'N/A', 'ote': 'ote.asdf', 'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf', 'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf', 'regions': 'N/A', 'dist

In [8]:
# Setup the test
slitx = [0] * 5
slity = [-.5, -.25, 0, .25, .5]
wave_range = [2.87e-06, 5.27e-06]
lam = np.array([2.9, 3.39, 3.88, 4.37, 5]) * 10**-6

order, wrange = nirspec.get_spectral_order_wrange(im, refs['wavelengthrange'])
im.meta.wcsinfo.sporder = order
im.meta.wcsinfo.waverange_start = wrange[0]
im.meta.wcsinfo.waverange_end = wrange[1]
# Use slice 0
slit_wcs = nirspec.nrs_wcs_set_input(im, 0)


In [9]:
# Slit to MSA entrance
# This includes the Slicer transform and the IFUFORE transform
slit2msa = slit_wcs.get_transform('slit_frame', 'msa_frame')
msax, msay, _= slit2msa(slitx, slity, lam)
print('slitx: ', slitx )
print('slity: ', slity)
print('msax: ', msax)
print('msay: ', msay)
print('diff slitx', slitx - t['xslitpos'])
print('diff slity', slity - t['yslitpos'])
print('diff x_msa', msax - t['xmsapos'])
print('diff y_msa', msay - t['ymaspos'])


slitx:  [0, 0, 0, 0, 0]
slity:  [-0.5, -0.25, 0, 0.25, 0.5]
msax:  [ 0.04131163  0.04131081  0.04131053  0.04131081  0.04131163]
msay:  [ -6.16116144e-04  -3.08195439e-04   3.09418172e-09   3.08202387e-04
   6.16125124e-04]
diff slitx xslitpos
--------
       0
       0
       0
       0
       0
diff slity yslitpos
--------
     0.0
     0.0
     0.0
     0.0
     0.0
diff x_msa      xmsapos     
-----------------
-0.00731033692549
 -0.0073085984888
-0.00730768015109
 -0.0073075728071
-0.00730828525063
diff y_msa      ymaspos     
-----------------
-0.00855614006785
-0.00854997507501
-0.00854340482798
-0.00853657871193
-0.00852965789751


In [10]:
# Slicer
slit2slicer = slit_wcs.get_transform('slit_frame', 'slicer')
x_slicer, y_slicer, _ = slit2slicer(slitx, slity, lam)
print('x_slicer: ', x_slicer)
print('y_slicer:', y_slicer)
print(x_slicer[0], y_slicer[0])

x_slicer:  [ 0.0004  0.0004  0.0004  0.0004  0.0004]
y_slicer: [-0.006 -0.003  0.     0.003  0.006]
0.000399999989895 -0.00600000005215


In [11]:
# MSA exit
# Applies the IFUPOST transform to coordinates at the Slicer
ifupost = datamodels.IFUPostModel('ifupost.asdf')
ifupost_transform = nirspec._create_ifupost_transform(ifupost.slice_0)
x_msa_exit, y_msa_exit = ifupost_transform(x_slicer, y_slicer, lam)
ifupost.close()
print('x_msa_exit: ', x_msa_exit)
print('y_msa_exit: ', y_msa_exit)
print('x_msa_exit: ', x_msa_exit)
print('y_msa_exit: ', y_msa_exit)
print('diff x_msa', x_msa_exit - t['xmsapos'])
print('diff y_msa', y_msa_exit - t['ymaspos'])

x_msa_exit:  [ 0.04862197  0.04861941  0.04861821  0.04861838  0.04861992]
y_msa_exit:  [ 0.00794002  0.00824178  0.00854341  0.00884478  0.00914578]
x_msa_exit:  [ 0.04862197  0.04861941  0.04861821  0.04861838  0.04861992]
y_msa_exit:  [ 0.00794002  0.00824178  0.00854341  0.00884478  0.00914578]
diff x_msa      xmsapos      
------------------
 5.20312959384e-13
-2.79785222768e-12
-7.32559846117e-13
-1.23519250383e-13
-2.09762762715e-14
diff y_msa      ymaspos      
------------------
-8.42248146227e-13
 9.06556080205e-13
 2.22133075822e-13
  1.2788832493e-12
 -1.8511737282e-12


In [12]:
# Computations are done using the eact form of the equations in the reports
from math import cos, sin
# Part I of the Forward IFU-POST transform - the linear transform
xc_out = 0.0487158154447 
yc_out = 0.00856211956976
xc_in = 0.000355277216
yc_in = -3.0089012e-05
theta = np.deg2rad(-0.129043957046)
factor_x = 0.100989874454
factor_y = 0.100405184145

# Slicer coordinates
xS = 0.000399999989895
yS = -0.00600000005215

x = xc_out + factor_x * (+cos(theta) * (xS - xc_in ) + sin(theta) * (yS - yc_in ))
y = yc_out + factor_y * (-sin(theta) * (xS - xc_in ) + cos(theta) * (yS - yc_in))

# Forward IFU-POST II part - non-linear transform

lam = 2.9e-6

# From the IFU-POST reference file for Slice 0, only the transform in y:

y_forw ={         
              'c0_0':-82.3492267824,
              'c0_1':29234.6982762,
              'c0_2':-540260.780853,
              'c0_3':771881.305018,
              'c0_4':-2563462.26848,
              'c0_5': 29914272.1164,
              'c1_0':4513.04082605,
              'c1_1':-2212869.44311,
              'c1_2':32875633.0303,
              'c1_3':-29923698.5288,
              'c1_4':27293902.5636,
              'c2_0':-39820.4434726,
              'c2_1':62431493.9962,
              'c2_2':-667197265.033,
              'c2_3':297253538.182,
              'c3_0':-1838860.86305,
              'c3_1':-777169857.2,
              'c3_2':4514693865.7,
              'c4_0':42790637.764,
              'c4_1':3596423850.94,
              'c5_0':-260274017.448,
              }
y_forw_dist = {
              'c0_0':188531839.97,
              'c0_1':-43453434864.0,
              'c0_2':70807756765.8,
              'c0_3':-308272809909.0,
              'c0_4':159768473071.0,
              'c0_5':9.71263334459e+12,
              'c1_0':-11762923852.9,
              'c1_1':3.54593887319e+12,
              'c1_2':-4.19864365542e+12,
              'c1_3':1.25456429831e+13,
              'c1_4':-1.17070515916e+13,
              'c2_0':173091230285.0,
              'c2_1':-1.08534069056e+14,
              'c2_2':8.28933480976e+13,
              'c2_3':-1.24708740989e+14,
              'c3_0':2.77438975799e+12,
              'c3_1':1.4767797203e+15,
              'c3_2':-5.45358301961e+14,
              'c4_0':-9.31015579941e+13,
              'c4_1':-7.53689063943e+15,
              'c5_0':6.46310545048e+14,
              }

y_coeff = {}
y_coeff['c0_0'] = y_forw['c0_0'] + lam * y_forw_dist['c0_0']
y_coeff['c0_1'] = y_forw['c0_1'] + lam * y_forw_dist['c0_1']
y_coeff['c0_2'] = y_forw['c0_2'] + lam * y_forw_dist['c0_2']
y_coeff['c0_3'] = y_forw['c0_3'] + lam * y_forw_dist['c0_3']
y_coeff['c0_4'] = y_forw['c0_4'] + lam * y_forw_dist['c0_4']
y_coeff['c0_5'] = y_forw['c0_5'] + lam * y_forw_dist['c0_5']
y_coeff['c1_0'] = y_forw['c1_0'] + lam * y_forw_dist['c1_0']
y_coeff['c1_1'] = y_forw['c1_1'] + lam * y_forw_dist['c1_1']
y_coeff['c1_2'] = y_forw['c1_2'] + lam * y_forw_dist['c1_2']
y_coeff['c1_3'] = y_forw['c1_3'] + lam * y_forw_dist['c1_3']
y_coeff['c1_4'] = y_forw['c1_4'] + lam * y_forw_dist['c1_4']
y_coeff['c2_0'] = y_forw['c2_0'] + lam * y_forw_dist['c2_0']
y_coeff['c2_1'] = y_forw['c2_1'] + lam * y_forw_dist['c2_1']
y_coeff['c2_2'] = y_forw['c2_2'] + lam * y_forw_dist['c2_2']
y_coeff['c2_3'] = y_forw['c2_3'] + lam * y_forw_dist['c2_3']
y_coeff['c3_0'] = y_forw['c3_0'] + lam * y_forw_dist['c3_0']
y_coeff['c3_1'] = y_forw['c3_1'] + lam * y_forw_dist['c3_1']
y_coeff['c3_2'] = y_forw['c3_2'] + lam * y_forw_dist['c3_2']
y_coeff['c4_0'] = y_forw['c4_0'] + lam * y_forw_dist['c4_0']
y_coeff['c4_1'] = y_forw['c4_1'] + lam * y_forw_dist['c4_1']
y_coeff['c5_0'] = y_forw['c5_0'] + lam * y_forw_dist['c5_0']


ifupost_y = y_coeff['c0_0'] + y_coeff['c0_1'] * y +y_coeff['c0_2'] *y**2 +y_coeff['c0_3'] *y**3+y_coeff['c0_4'] *y**4+y_coeff['c0_5']*y**5 \
+y_coeff['c1_0'] *x+y_coeff['c1_1'] *x*y +y_coeff['c1_2'] *x*y**2+y_coeff['c1_3'] *x*y**3+y_coeff['c1_4'] *x*y**4 \
+y_coeff['c2_0'] *x**2+y_coeff['c2_1'] *x**2*y+ y_coeff['c2_2']*x**2*y**2+ y_coeff['c2_3'] *x**2*y**3 \
+y_coeff['c3_0']*x**3 +y_coeff['c3_1'] *x**3*y+y_coeff['c3_2'] *x**3*y**2 \
+y_coeff['c4_0']*x**4  +y_coeff['c4_1'] *x**4*y \
+y_coeff['c5_0'] *x**5

print('ifupost_y', ifupost_y)
print('diff with table', ifupost_y - t['ymaspos'][0])
print('diff with pipeline', ifupost_y - y_msa_exit[0])

ifupost_y 0.007940023923424633
diff with table -8.52651282912e-13
diff with pipeline -1.04031366854e-14


In [13]:
# Reset lam - last cell changed it
lam = np.array([2.9, 3.39, 3.88, 4.37, 5]) * 10**-6

In [14]:
# Coordinates at Collimator exit
# Applies the Collimator forward transform to coordinates at the MSA exit
col = datamodels.open(refs['collimator'])
colx, coly = col.model.inverse(x_msa_exit, y_msa_exit)
col.close()
print('x_collimator_exit', colx)
print('y_collimator_exit', coly)

print('diff x_collimator_exit', colx - t['xcoll'])
print('diff y_collimator_exit', coly - t['ycoll'])

x_collimator_exit [ 0.07443997  0.07443401  0.07443022  0.07442859  0.07442913]
y_collimator_exit [-0.28242043 -0.28192622 -0.28143237 -0.28093908 -0.28044655]
diff x_collimator_exit       xcoll       
------------------
 8.25617352262e-13
-4.41598146939e-12
-1.15603360218e-12
-2.02671213145e-13
-2.15105711021e-14
diff y_collimator_exit       ycoll       
------------------
-1.39493971929e-12
 1.56807899998e-12
 3.85524945301e-13
   2.096656182e-12
-3.02829983312e-12


In [15]:
# After applying direcitonal cosines
from jwst.transforms import models as trmodels

dircos = trmodels.Unitless2DirCos()
xcolDircosi, ycolDircosi, z = dircos(colx, coly)
print('xcolDircosi ', xcolDircosi)
print('ycolDircosi ', ycolDircosi)
print('z', z)
print('diff xcolDircosi', xcolDircosi - t['xcolDirCosi'])
print('diff ycolDircosi', ycolDircosi - t['ycolDirCosi'])

xcolDircosi  [ 0.07145469  0.07145818  0.07146373  0.07147131  0.07148094]
ycolDircosi  [-0.27109449 -0.27065499 -0.27021559 -0.2697765  -0.26933787]
z [ 0.95989687  0.96002063  0.96014398  0.96026689  0.96038929]
diff xcolDircosi    xcolDirCosi    
------------------
   7.625150511e-13
-4.18866330509e-12
-1.09713627072e-12
-1.54792845208e-13
-7.65498775479e-14
diff ycolDircosi    ycolDirCosi    
------------------
-1.22524212998e-12
 1.31311628238e-12
 3.21631610234e-13
 1.86306525762e-12
-2.69778643869e-12


In [16]:
# Slit to GWA entrance
# applies the Collimator forward, Unitless to Directional and 3D Rotation to MSA exit coordinates
disp = datamodels.DisperserModel(refs['disperser'])
disperser = nirspec.correct_tilt(disp, im.meta.instrument.gwa_xtilt, im.meta.instrument.gwa_ytilt)
collimator2gwa = nirspec.collimator_to_gwa(refs, disperser)
x_gwa_in, y_gwa_in, z_gwa_in = collimator2gwa(x_msa_exit, y_msa_exit)
disp.close()
print('x_gwa_entrance:' , x_gwa_in)
print('y_gwa_entrance:' , y_gwa_in)
print('z_gwa_entrance:' , z_gwa_in)

print('diff x_gwa_entrance', x_gwa_in - t['xdispIn'])
print('diff y_gwa_entrance', y_gwa_in - t['ydispIn'])

2018-06-20 10:59:35,661 - stpipe - INFO - gwa_ytilt is 0.13448857 deg
2018-06-20 10:59:35,662 - stpipe - INFO - gwa_xtilt is 0.35986012 deg
2018-06-20 10:59:35,663 - stpipe - INFO - theta_y correction: -0.041645342310905185 deg
2018-06-20 10:59:35,664 - stpipe - INFO - theta_x correction: 0.0 deg


x_gwa_entrance: [ 0.22067237  0.22069163  0.22071285  0.22073601  0.22076113]
y_gwa_entrance: [-0.27015622 -0.26971665 -0.26927718 -0.26883799 -0.26839925]
z_gwa_entrance: [ 0.93718692  0.93730899  0.93743034  0.93755093  0.93767071]
diff x_gwa_entrance      xdispIn      
------------------
 7.00384195085e-13
-4.04309918878e-12
-1.05981889931e-12
-8.43769498715e-14
-1.71279657124e-13
diff y_gwa_entrance      ydispIn      
------------------
 -1.2196355037e-12
 1.28191901538e-12
 3.13471471003e-13
 1.86212156805e-12
-2.69861910596e-12


In [17]:
# Slit to GWA out
# Runs slit--> slicer --> msa_exit --> collimator --> dircos --> rotation --> angle_from_grating equation
slit2gwa = slit_wcs.get_transform('slit_frame', 'gwa')
x_gwa_out, y_gwa_out, z_gwa_out = slit2gwa(slitx, slity, lam)
print('x_gwa_exit:' , x_gwa_out)
print('y_gwa_exit:' , y_gwa_out)
print('z_gwa_exit:' , z_gwa_out)

print('diff x_gwa_exit', x_gwa_out - t['xdispLaw'])
print('diff y_gwa_exit', y_gwa_out - t['ydispLaw'])

x_gwa_exit: [ 0.03861286  0.08240387  0.12619291  0.16998001  0.22628237]
y_gwa_exit: [ 0.27015622  0.26971665  0.26927718  0.26883799  0.26839925]
z_gwa_exit: [ 0.96204192  0.95940739  0.95475921  0.948068    0.93635364]
diff x_gwa_exit      xdispLaw     
------------------
-7.00384195085e-13
 4.04309918878e-12
 1.05981889931e-12
 8.43769498715e-14
 1.71279657124e-13
diff y_gwa_exit      ydispLaw     
------------------
  1.2196355037e-12
-1.28191901538e-12
-3.13471471003e-13
-1.86212156805e-12
 2.69861910596e-12


In [18]:
# CAMERA entrance (assuming direction is from sky to detector)
angles = [disperser['theta_x'], disperser['theta_y'],
          disperser['theta_z'], disperser['tilt_y']]
rotation = models.Rotation3DToGWA(angles, axes_order="xyzy", name='rotation')
dircos2unitless = models.DirCos2Unitless()

gwa2cam = rotation.inverse | dircos2unitless
x_camera_entrance, y_camera_entrance = gwa2cam(x_gwa_out, y_gwa_out, z_gwa_out)
print('x_camera_entrance:' , x_camera_entrance)
print('y_camera_entrance:' , y_camera_entrance)

print('diff x_camera_enrtance', x_camera_entrance - t['xcamCosi'])
print('diff y_camera_entrance', y_camera_entrance - t['ycamCosi'])

x_camera_entrance: [-0.11311156 -0.06716992 -0.02134772  0.02464534  0.08453302]
y_camera_entrance: [ 0.28285903  0.28083671  0.27940655  0.27856425  0.27849854]
diff x_camera_enrtance      xcamCosi     
------------------
 -7.1256889278e-13
 4.19017598396e-12
 1.09577277807e-12
-2.33389696458e-14
  3.9673819785e-13
diff y_camera_entrance      ycamCosi     
------------------
 1.40382150349e-12
-1.55164769922e-12
-3.66373598126e-13
-2.08394412837e-12
  3.0342395263e-12


In [19]:
# at FPA
camera = datamodels.CameraModel(refs['camera'])
x_fpa, y_fpa = camera.model.inverse(x_camera_entrance, y_camera_entrance)
camera.close()
print('x_fpa: ', x_fpa )
print('y_fpa: ' , y_fpa)

print('diff x_fpa', x_fpa - t['xfpapos'])
print('diff y_fpa', y_fpa - t['yfpapos'])

x_fpa:  [-0.03260156 -0.0196373  -0.00661481  0.00648947  0.02349652]
y_fpa:  [-0.00354315 -0.0038227  -0.00402751 -0.00416094 -0.00418796]
diff x_fpa      xfpapos      
------------------
-1.93768612267e-13
  1.1848022563e-12
  3.1237686049e-13
 1.29323635134e-15
 8.94978535726e-14
diff y_fpa      yfpapos      
------------------
 3.75361634136e-13
-4.00640024628e-13
-9.63118473862e-14
 -5.6710278834e-13
 8.23793290528e-13


In [20]:
# at SCA
slit2sca = slit_wcs.get_transform('slit_frame', 'sca')
x_sca_nrs1, y_sca_nrs1 = slit2sca(slitx, slity, lam)

# At NRS2
fpa = datamodels.FPAModel(refs['fpa'])
x_sca_nrs2, y_sca_nrs2 = fpa.nrs2_model.inverse(x_fpa, y_fpa)
fpa.close()
print('x_sca1: ', x_sca_nrs1)
print('y_sca1: ' , y_sca_nrs1)
print('x_sca2: ', x_sca_nrs2 )
print('y_sca2: ' , y_sca_nrs2)

print('diff x_sca1', x_sca_nrs1[:3] - t['i'][:3])
print('diff y_sca1', y_sca_nrs1[:3] - t['j'][:3])
print('\n')
print('diff x_sca2', x_sca_nrs2[3:] - t['i'][3:])
print('diff y_sca2', y_sca_nrs2[3:] - t['j'][3:])

x_sca1:  [  309.40456252  1029.64117666  1753.11275396  2481.12822002  3425.96453057]
y_sca1:  [ 826.65806896  811.1275858   799.74961365  792.33653939  790.83550167]
x_sca2:  [ 3931.81779465  3211.58141255  2488.11000527  1760.09465002   815.25836199]
y_sca2:  [ 1220.24684926  1235.78808985  1247.17686774  1254.60081562  1256.11596538]
diff x_sca1        i       
---------------
 -1.00000001076
-0.999999934178
-0.999999982645
diff y_sca1        j       
---------------
-0.999999979147
 -1.00000002226
 -1.00000000535


diff x_sca2       i       
--------------
-1.00000000007
-1.00000000497
diff y_sca2        j       
---------------
-0.999999968494
 -1.00000004577


In [21]:
# at oteip
# Goes through slicer, ifufore, and fore transforms
slit2oteip = slit_wcs.get_transform('slit_frame', 'oteip')
x_oteip, y_oteip, _ = slit2oteip(slitx, slity, lam)
print('x_oteip: ', x_oteip)
print('y_oteip: ' , y_oteip)

print('diff x_oteip', x_oteip - t['xOTEIP'])
print('diff y_oteip', y_oteip - t['yOTEIP'])

x_oteip:  [ 0.05010543  0.05044043  0.05077637  0.05111296  0.05145047]
y_oteip:  [-0.04489819 -0.04451235 -0.04412679 -0.04374186 -0.04335772]
diff x_oteip       xOTEIP      
------------------
-6.93889390391e-18
-6.93889390391e-18
 1.38777878078e-17
-6.93889390391e-18
-2.08166817117e-17
diff y_oteip       yOTEIP      
------------------
 6.93889390391e-18
 6.93889390391e-18
-6.93889390391e-18
 6.93889390391e-18
 1.38777878078e-17


In [22]:
# at v2, v3 [in arcsec]
slit2v23 = slit_wcs.get_transform('slit_frame', 'v2v3')
v2, v3, _ = slit2v23(slitx, slity, lam)
v2 /= 3600
v3 /= 3600
print('v2: ', v2)
print('v3: ' , v3)

print('diff V2', v2 - t['xV2V3'])
print('diff V3', v3 - t['yV2V3'])

v2:  [ 0.08365329  0.08350689  0.08336007  0.08321298  0.08306549]
v3:  [-0.1387331  -0.13856511 -0.13839724 -0.13822965 -0.1380624 ]
diff V2       xV2V3       
------------------
 2.77555756156e-17
-1.38777878078e-17
-1.38777878078e-17
-1.38777878078e-17
               0.0
diff V3       yV2V3       
------------------
               0.0
               0.0
               0.0
-2.77555756156e-17
               0.0


In [23]:
# Save results to an asdf file

fa = AsdfFile()
fa.tree['slitx'] = list(slitx)
fa.tree['slity'] = list(slity)
fa.tree['lam'] = list(lam)
fa.tree['msax'] = list(msax)
fa.tree['msay'] = list(msay)
fa.tree['x_collimator_exit'] = list(colx)
fa.tree['y_collimator_exit'] = list(coly)
fa.tree['x_gwa_entrance'] = list(x_gwa_in)
fa.tree['y_gwa_entrance'] = list(y_gwa_in)
fa.tree['z_gwa_entrance'] = list(z_gwa_in)
fa.tree['x_gwa_exit'] = list(x_gwa_out)
fa.tree['y_gwa_exit'] = list(y_gwa_out)
fa.tree['z_gwa_exit'] = list(z_gwa_out)
fa.tree['x_camera_entrance'] = list(x_camera_entrance)
fa.tree['y_camera_entrance'] = list(y_camera_entrance)
fa.tree['x_fpa'] = list(x_fpa)
fa.tree['y_fpa'] = list(y_fpa)
fa.tree['x_sca_nrs1'] = list(x_sca_nrs1)
fa.tree['y_sca_nrs1'] = list(y_sca_nrs1)
fa.tree['x_sca_nrs2'] = list(x_sca_nrs2)
fa.tree['y_sca_nrs2'] = list(y_sca_nrs2)
fa.tree['x_oteip'] = list(x_oteip)
fa.tree['y_oteip'] = list(y_oteip)
fa.tree['v2'] = list(v2)
fa.tree['v3'] = list(v3)
fa.tree['x_slicer'] = list(x_slicer)
fa.tree['y_slicer'] = list(y_slicer)
fa.tree['x_msa_exit'] = list(x_msa_exit)
fa.tree['y_msa_exit'] = list(y_msa_exit)
fa.write_to("ifu_grating_functional.asdf", all_array_storage="internal")

In [24]:
im.meta.wcsinfo.waverange_end, im.meta.wcsinfo.waverange_start

(5.27e-06, 2.87e-06)

In [25]:
lam_cen = 0.5 * (im.meta.wcsinfo.waverange_end -
                 im.meta.wcsinfo.waverange_start
                ) + im.meta.wcsinfo.waverange_start
lam_cen

4.07e-06